In [0]:
from pyspark.sql import SparkSession
import pandas as pd
import random

# Initialize Spark Session
spark = SparkSession.builder.appName("Databricks_Catalog_Read").getOrCreate()

# Define the catalog and schema
catalog_name = "workspace"
schema_name = "default"

# Read data from Databricks tables and convert them to Pandas DataFrames

table_full_path = f"{catalog_name}.{schema_name}.{'pinterest_data'}"
    
# Read the table using Spark
df_spark = spark.read.table(table_full_path)
    
# Convert to Pandas DataFrame
globals()['df_pin'] = df_spark.toPandas()

In [0]:
# Show all rows
pd.set_option('display.max_rows', None)

In [0]:
df_pin

In [0]:
df_pin.replace('',None)
df_pin.replace(to_replace=r'.*User Info Error.*', value=None, regex=True, inplace=True)
df_pin.replace(to_replace=r'.*N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e.*', value=None, regex=True, inplace=True)
df_pin.replace(to_replace=r'.*No description available Story format.*', value=None, regex=True, inplace=True)
df_pin.replace(to_replace=r'.*No Title Data Available.*', value=None, regex=True, inplace=True)
df_pin.replace(to_replace=r'.*Image src error.*', value=None, regex=True, inplace=True)
df_pin = df_pin.applymap(lambda x: None if pd.isna(x) else x)
# Remove "Local save in " from the 'save_location' column
df_pin['save_location'] = df_pin['save_location'].str.replace('Local save in ', '', regex=False)



# Function to convert follower counts
def convert_to_number(num):
    if pd.isna(num):
        return None
    num = str(num).upper()  # Ensure it's uppercase
    if 'M' in num:
        return float(num.replace('M', ''))* 1000000
    elif 'K' in num:
        return float(num.replace('K', '')) * 1000
    elif 'B' in num:
        return float(num.replace('B', '')) * 1000000000
    else:
        return float(num) if num.isdigit() else None

# Apply conversion
#df_pin['index'] = df_pin['index'].apply(convert_to_number)
df_pin['follower_count'] = df_pin['follower_count'].apply(convert_to_number)

# Rename 'index' column to 'ind'
df_pin.rename(columns={'index': 'ind'}, inplace=True)
# Reorder columns in the specified order
df_pin = df_pin[['ind', 
                 'unique_id', 
                 'title', 
                 'description', 
                 'follower_count', 
                 'poster_name', 
                 'tag_list', 
                 'is_image_or_video', 
                 'image_src', 
                 'save_location', 
                 'category']]

In [0]:
df_pin

In [0]:
# Convert Pandas DataFrame to Spark DataFrame
df_pin_spark = spark.createDataFrame(df_pin)

#Save Spark DataFrame as managed delta table
df_pin_spark.write.mode("overwrite").saveAsTable("workspace.default.df_pinterest")